In [1]:
import os

In [2]:
%pwd

'c:\\Users\\gsaik\\OneDrive\\Desktop\\Project\\chicken_disease_classification_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\gsaik\\OneDrive\\Desktop\\Project\\chicken_disease_classification_project'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data = Path
    all_params: dict
    params_image_sise: list
    params_batch_size: int 
    

In [8]:
from src.CNN_Classifier.constant import *
from src.CNN_Classifier.utils.common import read_yaml,create_directory

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data = "artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_sise=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [10]:
from urllib.parse import urlparse